In [3]:
import pandas as pd

In [4]:
LOAN_AMT = 50000
LOAN_TERM = 60 #in month
APR = 0.1510 #in percentage

In [ ]:
# APR = (Interest / Loan amount) / Number of days in loan term)) x 365 x 100
# APR * N / 36500 = (I+F) / P

In [13]:
def calculate_interest(P,r):
    '''Calculate simple monthly interest payment
    '''
    
    return P*r/12
    
def calculate_snapshot_monthly(P, r, n):
    '''Calculate amortized monthly payment
    
    P (int): initial principal
    r (float): interest per period (e.g., APR/12)
    n (int): total number of payments
    '''
    r /= 12 #interest per month
    A = P / ( (1+r)**n-1 ) * (r*(1+r)**n)
    
    return A

def generate_amortization_table(P,r,n,additive=0):
    '''Generate complete amortization schedule
    
    P (int): initial principal
    r (float): interest per period (e.g., APR/12)
    n (int): total number of payments
    additive (float): additional monthly principal payment
    '''
    cols = ['Beginning Balance', 'Total Pmt', 'Interest', 'Principal', 'Ending Balance']
    df = pd.DataFrame(columns=cols)
    M = calculate_snapshot_monthly(P=P, r=r, n=n) #monthly total (this is constant)
    
    for i in range(n):
        I = calculate_interest(P=P, r=r)
        P_i = P #initial balance
        pr = M-I #principal
        P -= pr+additive #ending balance

        if P < 0: P = 0

        tmp = [P_i, M, I, pr, P]
        tmp = [round(j,2) for j in tmp]
        
        df = pd.concat([df, pd.DataFrame(tmp, index=cols).T])

        if P == 0: #break cycle if ends early
            break

    df = df.reset_index(drop=True)
    df.index += 1
    
    return df

In [15]:
generate_amortization_table(P=LOAN_AMT, r=APR, n=LOAN_TERM, additive=150)

,Beginning Balance,Total Pmt,Interest,Principal,Ending Balance
1,50000.0,1192.12,629.17,562.96,49287.04
2,49287.04,1192.12,620.2,571.93,48565.12
3,48565.12,1192.12,611.11,581.01,47834.1
4,47834.1,1192.12,601.91,590.21,47093.89
5,47093.89,1192.12,592.6,599.52,46344.37
6,46344.37,1192.12,583.17,608.96,45585.41
7,45585.41,1192.12,573.62,618.51,44816.91
8,44816.91,1192.12,563.95,628.18,44038.73
9,44038.73,1192.12,554.15,637.97,43250.76
10,43250.76,1192.12,544.24,647.88,42452.88


In [90]:
calculate_snapshot_monthly(P=LOAN_AMT, r=APR, n=LOAN_TERM)

1192.122677298203

In [62]:
calculate_interest(P=LOAN_AMT, r=APR)

625.0

In [64]:
generate_amortization_table(P=LOAN_AMT, r=APR, n=LOAN_TERM)

564.4965043179393

In [44]:
LOAN_AMT / ( (1+APR)**LOAN_TERM-1 ) * (APR*(1+APR)**LOAN_TERM)

1189.4965043179393

In [7]:
(50000-I)*0.15/12

617.1875